# Deploying ML model as Public API using FastAPI and Ngrok
This project demonstrates how machine learning models can be effectively deployed as a web service and how these services can be accessed over the internet using tools like ngrok.

## Project Overview
- ### FastAPI Server - Backend
  - _FastAPI:_ The backend is built using FastAPI, which is a modern, fast web framework for building APIs with Python.
  - _Model and Scaler:_ The backend loads a pre-trained machine learning model and a scaler (used for feature normalization) from disk using the pickle library.
  - _Input Data Model:_ The data required for prediction is defined using Pydantic’s BaseModel. This includes features such as Pregnancies, Glucose, BloodPressure, etc.
  - _Prediction Endpoint:_ A POST endpoint /diabetes_prediction is created to accept input data, scale it, and then make a prediction using the loaded model. The prediction result is either "The person is not diabetic" or "The person is diabetic".
    - #### Imports and Setup
      - _FastAPI:_ A web framework to build APIs quickly with Python.
      - _Pydantic's BaseModel:_ Used to define the input data model with validation.
      - _Pickle:_ For loading the pre-trained machine learning model and the scaler.
      - _Uvicorn:_ ASGI server to run the FastAPI app.
      - _Pyngrok:_ To create a public URL for the local server using ngrok.
      - _CORSMiddleware:_ Middleware to handle Cross-Origin Resource Sharing, allowing the API to be accessed from different domains.
      - _Nest_asyncio:_ Allows running asynchronous event loops within Jupyter notebooks or environments that normally do not allow it.
    - #### CORS Configuration
      - The _origins_ variable is set to _["*"]_, meaning the API will allow requests from any origin.
      - The _CORSMiddleware_ is added to the FastAPI application to handle CORS, allowing the API to be accessible from different domains and allowing various HTTP methods and headers.
    - #### Input Data Model
      - The _diabetes_input_ class inherits from _BaseModel_ and defines the structure of the input data. It includes features like _Pregnancies, Glucose, BloodPressure_, etc., that are required for making a prediction.
    - #### Loading the Model and Scaler
      - The pre-trained diabetes prediction model and the scaler (used to standardize the input data) are loaded using the _pickle_ library. These are saved models likely trained previously on a diabetes dataset.
    - #### Prediction Endpoint
      - The _@app.post('/diabetes_prediction')_ decorator defines an endpoint that listens for POST requests at _/diabetes_prediction_.
      - The _diabetes_pred_ function takes the input data, converts it into a dictionary, extracts the relevant features, scales the input data using the loaded scaler, and then makes a prediction using the loaded model.
      - The prediction is returned as either **"The person is not diabetic"** or **"The person is diabetic"** based on the model's output.
    - #### ngrok Setup
      - The _authtoken_ is used to authenticate with ngrok.
      - A tunnel is created on port 8000, and a public URL is printed, which can be used to access the local FastAPI application from anywhere on the web.
    - #### Run the Application
      - _nest_asyncio.apply()_ is used to ensure that the async event loop can run in environments that normally wouldn't support it (like Jupyter notebooks).
      - Finally, _uvicorn.run(app, port=8000)_ starts the FastAPI application on port 8000.


In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=125214 sha256=9461de943007c533405401d6b76fa394fd447c582e8d66faf0513f76af744349
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.7 MB/s eta 0:00:00


In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [3]:
app = FastAPI()

In [4]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins = origins,
    allow_credentials = True,
    allow_methods = ["*"],
    allow_headers = ["*"],
)

In [5]:
class diabetes_input(BaseModel):
    
    Pregnancies: int
    Glucose: int
    BloodPressure: int
    SkinThickness: int
    Insulin: int
    BMI: float
    DiabetesPedigreeFunction: float
    Age: int

In [6]:
# Loading the saved model
diabetes_model = pickle.load(open('/kaggle/input/diabetes-model-sav/diabetes_model.sav', 'rb'))

In [7]:
# Loading the saved scaler
diabetes_scaler = pickle.load(open('/kaggle/input/diabetes-model-sav/diabetes_scaler.sav', 'rb'))

In [8]:
@app.post('/diabetes_prediction')
def diabetes_pred(input_parameters: diabetes_input):
    input_data = input_parameters.dict()  # Convert to dictionary directly

    pregnancies = input_data['Pregnancies']
    glucose = input_data['Glucose']
    bloodpressure = input_data['BloodPressure']
    skinthickness = input_data['SkinThickness']
    insulin = input_data['Insulin']
    bmi = input_data['BMI']
    diabetespedigreefunction = input_data['DiabetesPedigreeFunction']
    age = input_data['Age']

    input_list = [pregnancies, glucose, bloodpressure, skinthickness, insulin, bmi, diabetespedigreefunction, age]

    # Scale the input data
    scaled_input = diabetes_scaler.transform([input_list])

    # Make prediction
    prediction = diabetes_model.predict(scaled_input)

    if prediction[0] == 0:
        return 'The person is not diabetic'
    else:
        return 'The person is diabetic'

In [9]:
authtoken = "2kPrVIfzpXxqzoI2puYjKh55HUP_74GR5NpqtaE29zXhTPHaU"  
ngrok.set_auth_token(authtoken)

In [10]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [33]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://c84d-35-201-181-175.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [33]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._ru